In [1]:
import numpy as np
import pandas as pd
import itertools


In [2]:
def freeze(struct):
    return frozenset(map(frozenset, struct))

def unfreeze(frozen_struct):
    return list(map(list, frozen_struct))

def remove_players(coalition, players_to_remove):
    """
    Removes all players from coalition that are in players_to_remove
    :param coalition: 
    :param players_to_remove: 
    :return: The coalition without the removed players
    """
    
    return [p for p in coalition if p not in players_to_remove]


In [12]:
class Structure:
    def __init__(self, frozen_struct):
        self.struct = frozen_struct
    
    def __str__(self):
        return str(unfreeze(self.struct))
    
    def __repr__(self):
        return str(unfreeze(self.struct))
    
    def __hash__(self):
        return self.struct.__hash__()
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()
    
    def move_coalition(self, coalition):
        """
        Moves all players in coalition from their orginal coalition to form a new one.
        :param coalition: New coalition to form 
        :return: The new coalition structure
        """
        structure = unfreeze(self.struct)
        new_structure = [remove_players(c, coalition) for c in structure]
        filtered = list(filter(None, new_structure))
        return Structure(freeze(filtered + [coalition]))
    
    def check_blocking_coalition(self, dic, coalition):
        """
        checks whether the given coalition blocks the structure
        :param dic: Dictionary containing all utilities for all structures
        :param coalition: The coalition to check
        :return: True if the coalition actually blocks the structure
        """
        new_structure = self.move_coalition(coalition)
        res = True
        
        for p in coalition:
            if dic[self][p] >= dic[new_structure][p]:
                res = False
                break
                
        return res
    
    def is_core_stable(self, dic, all_cs):
        """
        checks whether the structure is core stable
        :param dic: Dictionary containing all utilities for all structures
        :param all_cs: List of all possible coalitions
        :return: True if the structure is core stable
        """
        for c in all_cs:
            if self.check_blocking_coalition(dic, c):
                return False
        return True
        

In [9]:
# https://stackoverflow.com/questions/19368375/set-partitions-in-python

def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller



In [10]:
def find_coalition(partition, player):

    return [c for c in partition if player in c][0]

def value_coalition(coalition, n, player, friends):
    v = 0
    for p in coalition:
        if p == player:
            continue
        elif p in friends[player]:
            v += n
        else:
            v -= 1
    return v
     
def utility(partition, n, player, friends, type='SF'):
    M = n**2
    
    c = find_coalition(partition, player)
    own_value = value_coalition(c, n, player, friends)
    min_value_friends = min([value_coalition(find_coalition(partition, friend), n, friend, friends) 
                             for friend in friends[player]])
    if type=='SF':
        return M * own_value \
               + min_value_friends
    
    if type=='EQ':
        return min([own_value, min_value_friends])
    
    if type=='AL':
        return own_value \
               + M * min_value_friends



In [6]:
def calculate_all_utilities(N, F, degree='SF'):
    n = len(N)
    partitions = enumerate(partition(list(N)), 1)
    dic = dict()
    
    for i, p in partitions:
        uts = []
        for player in N:
            uts.append(utility(p, n, player, F, type=degree))
        dic[Structure(freeze(p))] = uts
    return dic
    

        

In [8]:
n = 6
N = range(n)
F = {
    0: [1,2],
    1: [0,2],
    2: [0,1,3],
    3: [2,4,5],
    4: [3,5],
    5: [3,4]
}


In [14]:
dic = calculate_all_utilities(N, F)



In [7]:
def find_all_coalitions(N):
    res = []
    for m in range(len(N)-1):
        res += (list(itertools.combinations(N, m+1)))
    return res

In [12]:
all_cs = find_all_coalitions(N)
all_cs


[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 3, 4),
 (0, 3, 5),
 (0, 4, 5),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5),
 (0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 2, 5),
 (0, 1, 3, 4),
 (0, 1, 3, 5),
 (0, 1, 4, 5),
 (0, 2, 3, 4),
 (0, 2, 3, 5),
 (0, 2, 4, 5),
 (0, 3, 4, 5),
 (1, 2, 3, 4),
 (1, 2, 3, 5),
 (1, 2, 4, 5),
 (1, 3, 4, 5),
 (2, 3, 4, 5),
 (0, 1, 2, 3, 4),
 (0, 1, 2, 3, 5),
 (0, 1, 2, 4, 5),
 (0, 1, 3, 4, 5),
 (0, 2, 3, 4, 5),
 (1, 2, 3, 4, 5)]

In [17]:
n = 6
N = range(n)
F = {
    0: [1,2],
    1: [0,2],
    2: [0,1,3],
    3: [2,4,5],
    4: [3,5],
    5: [3,4]
}

dic_sf = calculate_all_utilities(N, F, degree='SF')
dic_eq = calculate_all_utilities(N, F, degree='EQ')
dic_al = calculate_all_utilities(N, F, degree='AL')

all_cs = find_all_coalitions(N)
gamma = Structure(freeze([[0,1,2,3,4,5]]))

print('SF', gamma.is_core_stable(dic_sf, all_cs))
print('EQ', gamma.is_core_stable(dic_eq, all_cs))
print('AL', gamma.is_core_stable(dic_al, all_cs))
    







SF True
EQ False
AL False


In [19]:
gamma = Structure(freeze([[0],[1,2,3,4,5]]))
delta = Structure(freeze([[0,1,2],[3,4,5]]))

print(gamma)
print(all_cs[0])
print(gamma.move_coalition(all_cs[1]))



[[1, 2, 3, 4, 5], [0]]
(0,)
[[0], [1], [2, 3, 4, 5]]


In [169]:
t = [[1,3,2], [0,20], [7,8,9,10,11]]
p = [[20,0], [2,3], [7,8,9,10,11], [1]]

ft = freeze(t)
fp = freeze(p)

st = Structure(ft)
pt = Structure(fp)

mt = st.move_coalition([2,3])

print(pt == mt)
mt


True


[[1], [7, 8, 9, 10, 11], [0, 20], [2, 3]]

In [13]:
l = [[], [1,2,3,4]]



[None, [1]]